Inainte sa trimiti notebook-ul:

* asigura-te ca totul se executa cum trebuie. Prima oara, **restarteaza kernel-ul** (in meniu, selecteaza Kernel$\rightarrow$Restart & Run all);

* nu lasa niciun mesaj la output (`disp`, `printf`, linii care nu sunt incheiate cu `;`) in functiile pe care le scrii decat daca ti se specifica explicit sa faci acest lucru! Orice abatere va fi **depunctata automat** de grader;

* citeste instructiunile de [aici](https://github.com/radusqrt/numerical-methods/blob/master/lab_instructions.md) cu atentie;

* asigura-te ca rezolvi toate celulele in care scrie `YOUR CODE HERE` sau `YOUR ANSWER HERE` si ca iti treci numele mai jos:

In [1]:
STUDENT_NAME = "Ispas Alexandra-Petrina";

---

# Laborator 4: Eliminari gaussiene
### Autor: Radu Stochitoiu

## Functii ajutatoare    

In [2]:
function [res] = isequal(A, B, err)
    res = norm(A - B) < err;
endfunction

function [x] = SST(A, b)
    n = length(b);
    x = [b(n) / A(n, n)];
    for i = n-1:-1:1
        x = [(b(i) - A(i, [i+1:n]) * x) / A(i,i); x];
    endfor
endfunction

In [1]:
A = [1 2 3;1 2 3;1 1 1 ; 2 2 1]
det(A)

A =

   1   2   3
   1   2   3
   1   1   1
   2   2   1

error: det: A must be a square matrix


## Problema 1

Implementati functia ```G(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana.

In [3]:
function [R, x] = G(A, b)
    n = length(A);

    for p = 1:n-1
        for i = p+1:n
            if A(p, p) == 0
                continue;
            endif
            b(i) = b(i) - b(p)*(A(i, p)/A(p, p));
            A(i, :) = A(i, :) - A(p, :)*(A(i, p)/A(p,p));
        endfor
    endfor
    R = A;
    x = SST(A,b);
endfunction


In [4]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [2 1 7; 0 -0.5 -10.5; 0 0 18];
[R, x] = G(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 2

Implementati functia ```GPP(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala.

In [5]:
function [R, x] = GPP(A, b)
    n = length(A);
    
    for p = 1:n-1
        piv = -inf;
        lin = -1;
        for i = p:n
            if piv < abs(A(i, p))
                piv = A(i,p);
                lin = i;
            endif
        endfor
        
        if p ~= lin
            for j = p : n
                aux = A(p, j);
                A(p,j) = A(lin, j);
                A(lin, j) = aux;
            endfor
            aux = b(lin);
            b(lin) = b(p);
            b(p) = aux;
        endif
    endfor
  [R, x] = G(A, b);
endfunction

In [6]:
% Test 1
A = [0 1 3; 2 1 3; -3 1 1];
b = [3 1 1]';
R_ref = [-3 1 1; 0 5/3 11/3; 0 0 4/5];
[R, x] = GPP(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 3

Implementati functia ```GPPS(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala scalata.

In [7]:
function [R, x] = GPPS(A, b)
   n = length(A);
   max = zeros(n, 1);
   lin = -1;
   
   for p = 1 : n - 1
   
       for i = p : n
           T = zeros(n, 1);
           s = zeros(n, 1);
           
           for j = p : n
               if s(i) < abs(A(i, j))
                   s(i) = abs( A(i, j));
                endif
            endfor
        
            T(i) = abs(A(i, p)) / s(i);
            
            if T(i) > max(p)
                max(p) = T(i);
                lin = i;
            endif      
        endfor
        
        if p ~= lin
            for i = p : n
                aux = A(p, i);
                A(p,i) = A(lin, i);
                A(lin, i) = aux;
            endfor
            aux = b(lin);
            b(lin) = b(p);
            b(p) = aux;
        endif
        for i = p+1:n
             b(i) = b(i) - b(p)*(A(i, p)/A(p, p));
            A(i, :) = A(i, :) - A(p, :)*(A(i, p)/A(p,p));
        endfor
    endfor
    R = A;
    x = SST(A, b);
endfunction

In [8]:
% Test 1
A = [3 -13 9 3; -6 4 1 -18; 6 -2 2 4; 12 -8 6 10];
b = [-19 -34 16 26]';
R_ref = [6 -2 2 4; 0 -12 8 1; 0 0 -2/3 5/3; 0 0 0 -3];
[R, x] = GPPS(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 4

Implementati functia ```GPT(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare totala (se pastreaza ordinea initiala a elementelor lui $x$).

In [9]:
function [R, x] = GPT(A, b)
    
    n = length(A);
    c1 = zeros(n,1);
    c2 = zeros(n,1);
    y = zeros(1,n);
    k = 1; q = 1;
    for p = 1 : n-1
    
        piv = -inf;
        lin = -1;
        col = -1;
        
        for i = p : n
            for j = p : n
                if piv < abs(A(i, j))
                    piv = abs(A(i, j));
                    lin = i;
                    col = j;
                endif
            endfor
        endfor
        
        if p ~= lin
        
            for j = p : n
            
                aux = A(p, j);
                A(p, j) = A(lin, j);
                A(lin, j) = aux;
                
            endfor
            aux = b(p);
            b(p) = b(lin);
            b(lin) = aux;
        endif
            
        if p ~= col
            c1(k++) = p;
            c2(q++) = col;
            for i = 1 : n
                aux = A(i, p);
                A(i, p) = A(i, col);
                A(i, col) = aux;
            endfor
            
        endif
        for i = p+1:n
            if A(i, p) == 0
                for j = p:n
                    aux = A(i,j);
                    A(i,j) = A(end,j);
                    A(end,j) = aux;
                endfor
                aux = b(i);
                b(i) = b(end);
                b(end) = aux;
            endif
        endfor
     
        for i = p+1:n
             b(i) = b(i) - b(p)*(A(i, p)/A(p, p));
            A(i, :) = A(i, :) - A(p, :)*(A(i, p)/A(p,p));
        endfor
        
    endfor
    R = A;
    x = SST(A, b);
    for k = n:-1:1
        if c1(k) ~= 0
            aux = x(c1(k));
            x(c1(k)) = x(c2(k));
            x(c2(k)) = aux;
        endif
    endfor
endfunction

In [10]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [7 2 1; 0 27/7 3/7; 0 0 2/3];
[R, x] = GPT(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 5 (BONUS)

Implementati functia ```GaussJordan(A)``` ce primeste ca parametru o matrice $A$ patratica, de lungime pozitiva, si returneaza inversa lui $A$ calculata prin procedeul Gauss-Jordan.

In [1]:
function [A] = Gbonus(A)
    [n m] = size(A);
    if A(1,1) ~= 1
        A(1,:) = A(1,:)/A(1,1);
    endif
    for p = 1:n-1
        for i = p+1:n
            if A(p, p) == 0
                continue;
            endif
            A(i, :) = A(i, :) - A(p, :)*(A(i, p)/A(p,p));
        endfor
    endfor
    for i = 1:n
         A(i,:) = A(i,:)/A(i,i);
    endfor
    for p = n:-1:2
        for i = p-1:-1:1
             A(i, :) = A(i, :) - A(p, :)*(A(i, p)/A(p,p));
        endfor
    endfor
endfunction

In [3]:
A = [2 3 0; 1 -2 -1; 2 0 -1];

Gbonus([A, eye(3)])

ans =

   1.00000   0.00000   0.00000   2.00000   3.00000  -3.00000
   0.00000   1.00000   0.00000  -1.00000  -2.00000   2.00000
  -0.00000  -0.00000   1.00000   4.00000   6.00000  -7.00000



In [13]:
function [invA] = GaussJordan(A)
    n = length(A);
    X = [A, eye(n)];
    B = Gbonus(X);
    [a b] = size(B);
    invA = B(:,a+1:b);
endfunction

In [14]:
% Test 1
A = [2 3 0; 1 -2 -1; 2 0 -1];
[invA] = GaussJordan(A);
assert(isequal(invA, inv(A), 1e-5));